NOT ADVISABLE 😂

In [ ]:


import tensorflow as tf
import numpy as np
import os
from sklearn.model_selection import train_test_split
import librosa
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models

# Function to extract features from audio files
def extract_features(audio_path):
    # Load the audio file
    y, sr = librosa.load(audio_path, sr=None)

    # Extract MFCC (Mel-frequency cepstral coefficients) features
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

    # Calculate the mean of MFCCs across all frames
    mfccs_mean = np.mean(mfccs, axis=1)

    return mfccs_mean

# Define a function to load audio data and labels from three directories
def load_audio_data(data_dir):
    data = []
    labels = []

    # Map directory names to labels
    label_mapping = {
        'good': 'Good Engine',
        'bad': 'Bad Engine',
        'background_noise': 'Machine Stopped'
    }

    for label_dir, label in label_mapping.items():
        label_dir = os.path.join(data_dir, label_dir)
        for file in os.listdir(label_dir):
            if file.endswith(".wav"):
                audio_path = os.path.join(label_dir, file)
                features = extract_features(audio_path)
                data.append(features)
                labels.append(label)

    return np.array(data), np.array(labels)

# Load audio data and labels
data_dir = "C:\Users\Salifu Hamdia\Downloads\New folder (2)\sound" 
data, labels = load_audio_data(data_dir)

# Split data into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.15, random_state=42)

# Encode labels into numerical values
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Create a simple neural network model
num_features = len(train_data[0])
model = models.Sequential([
    layers.Input(shape=(num_features,)),  # Input layer with the number of features
    layers.Dense(64, activation='relu'),  # Hidden layer with 64 neurons and ReLU activation
    layers.Dense(32, activation='relu'),  # Hidden layer with 32 neurons and ReLU activation
    layers.Dense(3, activation='softmax')  # Output layer with 3 neurons (one for each class) and softmax activation
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 50
model.fit(train_data, train_labels_encoded, epochs=epochs, validation_split=0.1, verbose=2)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_data, test_labels_encoded, verbose=2)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

# Save the trained model as "soundclassifier_with_metadata.tflite"
model.save("soundclassifier_with_metadata.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("soundclassifier_with_metadata.tflite", "wb") as f:
    f.write(tflite_model)
